## 載入函式庫

In [1]:
import tensorflow as tf
from keras_segmentation.models.unet import resnet50_unet
import keras
import numpy as np
import os
import cv2
import time
import matplotlib.pyplot as plt

Using TensorFlow backend.


## 解決 TensorFlow 2.0 程式出現 cuDNN failed to initialize 錯誤問題

https://davistseng.blogspot.com/2019/11/tensorflow-2.html

In [2]:
def solve_cudnn_error():
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            # Currently, memory growth needs to be the same across GPUs
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
        except RuntimeError as e:
            # Memory growth must be set before GPUs have been initialized
            print(e)

solve_cudnn_error()

1 Physical GPUs, 1 Logical GPUs


## 第一次建模

資料集：DATA_0811_1
* **腫瘤視為背景**
* 去除case180、203，重新調整case160的影像大小
* 沒有做退縮
* WW = 600, WL = 100
* Base Model: Resnet-50
* Segmentation Model: U-Net

In [3]:
result_list = [] # 用來記錄各次建模的結果

### 查看模型架構

In [4]:
model = resnet50_unet(n_classes = 2, input_height = 512, input_width = 512)
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 518, 518, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 256, 256, 64) 9472        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 256, 256, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

### 模型參數設定

In [5]:
dataset_dir = 'DATA_0811_1\\WW600WL100'
input_size = 512
epochs = 100
steps_per_epoch = 512

### 訓練及評估模型

目前用來評估模型表現的指標：
* average dice score per case
* average recall
* average precision
* global dice score

In [6]:
model_dir = os.path.join(dataset_dir, 'model_resnet50_unet')

if not os.path.exists(model_dir):
    os.makedirs(model_dir)
    print('-----建立新資料夾：' + model_dir + '-----') 

st = time.time()

target_dataset = dataset_dir
print(f'########## 即將用該資料集進行訓練：{target_dataset} ##########')

model_name = 'resnet50_unet_padding0'
checkpoints_path = os.path.join(model_dir, model_name)

keras.backend.clear_session() # 銷毀當前的TF，避免舊模型/網路層混亂，並減緩內存
model = resnet50_unet(n_classes = 2, input_height = input_size, input_width = input_size)

model.train(
    train_images = os.path.join(dataset_dir, 'train', 'images'),
    train_annotations = os.path.join(dataset_dir, 'train', 'annotations_kidney only'),
    epochs = epochs, # default = 5
    steps_per_epoch = steps_per_epoch, # default = 512
    verify_dataset = False, # 當資料量很多時檢驗資料集會花很多時間
    checkpoints_path = checkpoints_path
)

ed = time.time()
spend_time = ed - st
print('花費時間(秒)：' + str(spend_time))
print()

-----建立新資料夾：DATA_0811_1\WW600WL100\model_resnet50_unet-----
########## 即將用該資料集進行訓練：DATA_0811_1\WW600WL100 ##########
Epoch 1/100
512/512 [==============================] - 183s 357ms/step - loss: 0.0932 - accuracy: 0.9784
saved  DATA_0811_1\WW600WL100\model_resnet50_unet\resnet50_unet_padding0.0
Epoch 2/100
512/512 [==============================] - 148s 289ms/step - loss: 0.0073 - accuracy: 0.9978
saved  DATA_0811_1\WW600WL100\model_resnet50_unet\resnet50_unet_padding0.1
Epoch 3/100
512/512 [==============================] - 148s 289ms/step - loss: 0.0066 - accuracy: 0.9979
saved  DATA_0811_1\WW600WL100\model_resnet50_unet\resnet50_unet_padding0.2
Epoch 4/100
512/512 [==============================] - 146s 285ms/step - loss: 0.0052 - accuracy: 0.9984
saved  DATA_0811_1\WW600WL100\model_resnet50_unet\resnet50_unet_padding0.3
Epoch 5/100
512/512 [==============================] - 145s 284ms/step - loss: 0.0044 - accuracy: 0.9985
saved  DATA_0811_1\WW600WL100\model_resnet50_unet\resnet50

512/512 [==============================] - 145s 283ms/step - loss: 0.0014 - accuracy: 0.9995
saved  DATA_0811_1\WW600WL100\model_resnet50_unet\resnet50_unet_padding0.44
Epoch 46/100
512/512 [==============================] - 145s 282ms/step - loss: 0.0011 - accuracy: 0.9996
saved  DATA_0811_1\WW600WL100\model_resnet50_unet\resnet50_unet_padding0.45
Epoch 47/100
512/512 [==============================] - 145s 283ms/step - loss: 0.0014 - accuracy: 0.9995
saved  DATA_0811_1\WW600WL100\model_resnet50_unet\resnet50_unet_padding0.46
Epoch 48/100
512/512 [==============================] - 145s 283ms/step - loss: 0.0012 - accuracy: 0.9995
saved  DATA_0811_1\WW600WL100\model_resnet50_unet\resnet50_unet_padding0.47
Epoch 49/100
512/512 [==============================] - 145s 283ms/step - loss: 0.0010 - accuracy: 0.9996
saved  DATA_0811_1\WW600WL100\model_resnet50_unet\resnet50_unet_padding0.48
Epoch 50/100
512/512 [==============================] - 145s 282ms/step - loss: 0.0011 - accuracy: 0.99

512/512 [==============================] - 145s 283ms/step - loss: 8.3391e-04 - accuracy: 0.9997
saved  DATA_0811_1\WW600WL100\model_resnet50_unet\resnet50_unet_padding0.89
Epoch 91/100
512/512 [==============================] - 145s 283ms/step - loss: 7.7348e-04 - accuracy: 0.9997
saved  DATA_0811_1\WW600WL100\model_resnet50_unet\resnet50_unet_padding0.90
Epoch 92/100
512/512 [==============================] - 145s 283ms/step - loss: 8.7102e-04 - accuracy: 0.9997
saved  DATA_0811_1\WW600WL100\model_resnet50_unet\resnet50_unet_padding0.91
Epoch 93/100
512/512 [==============================] - 145s 283ms/step - loss: 8.4142e-04 - accuracy: 0.9997
saved  DATA_0811_1\WW600WL100\model_resnet50_unet\resnet50_unet_padding0.92
Epoch 94/100
512/512 [==============================] - 145s 282ms/step - loss: 8.9667e-04 - accuracy: 0.9996
saved  DATA_0811_1\WW600WL100\model_resnet50_unet\resnet50_unet_padding0.93
Epoch 95/100
512/512 [==============================] - 145s 283ms/step - loss: 8.3

In [8]:
from evaluate import *

st = time.time()

result = evaluate_model(
    image_dir = os.path.join(dataset_dir, 'test', 'images'), 
    label_dir = os.path.join(dataset_dir, 'test', 'annotations_kidney only'), 
    checkpoints_path = 'DATA_0811_1\\WW600WL100\\model_resnet50_unet\\resnet50_unet_padding0',
    calculate_predicting_indicators = True,
    output_predicted_result = False, 
    segment_out_predicted_region_from_original_images = False, 
    roi_name = 'kidney', 
    preds = None)

print('訓練集預測結果：')
print(f'average Dice score per case of kidney: {result[0]: .4f}')
print(f'average recall of kidney: {result[1]: .4f}')
print(f'average precision of kidney: {result[2]: .4f}')
print(f'global dice score of kidney: {result[3]: .4f}')
result_list.append(result)

ed = time.time()
spend_time = ed - st
print('花費時間(秒)：' + str(spend_time))
print()

----------生成模型預測結果----------
loaded weights  DATA_0811_1\WW600WL100\model_resnet50_unet\resnet50_unet_padding0.99


100%|████████████████████████████████████████████████████████████████████████████| 12656/12656 [13:38<00:00, 15.46it/s]


----------開始計算各項預測指標----------
目前進度：第500張照片
目前進度：第1000張照片
目前進度：第1500張照片
目前進度：第2000張照片
目前進度：第2500張照片
目前進度：第3000張照片
目前進度：第3500張照片
目前進度：第4000張照片
目前進度：第4500張照片
目前進度：第5000張照片
目前進度：第5500張照片
目前進度：第6000張照片
目前進度：第6500張照片
目前進度：第7000張照片
目前進度：第7500張照片
目前進度：第8000張照片
目前進度：第8500張照片
目前進度：第9000張照片
目前進度：第9500張照片
目前進度：第10000張照片
目前進度：第10500張照片
目前進度：第11000張照片
目前進度：第11500張照片
目前進度：第12000張照片
目前進度：第12500張照片
total case number: 12656
訓練集預測結果：
average Dice score per case of kidney:  0.9443
average recall of kidney:  0.9423
average precision of kidney:  0.9477
global dice score of kidney:  0.9479
花費時間(秒)：1514.3356366157532



In [ ]:
# st = time.time()

# preds = result[4]

# _ = evaluate_model(
#     image_dir = os.path.join(dataset_dir, 'test', 'images'), 
#     label_dir = os.path.join(dataset_dir, 'test', 'annotations_kidney only'), 
#     checkpoints_path = 'DATA_0811_1\\WW600WL100\\\model_resnet50_unet\\resnet50_unet_padding0',
#     calculate_predicting_indicators = False,
#     output_predicted_result = True, 
#     segment_out_predicted_region_from_original_images = True, 
#     roi_name = 'kidney', 
#     preds = preds
# )

# ed = time.time()
# spend_time = ed - st
# print('花費時間(秒)：' + str(spend_time))
# print()

## 隨機印出幾張CT影像的預測結果(訓練集)

由於之後要把訓練集中的腎臟影像作為第二階段的輸入，因此這裡是印出訓練集的預測結果(測試集在這裡只是評估模型是否能夠準確預測腎臟區域)

In [ ]:
# from evaluate import *
# show_result(
#     target_dataset_base_dir = 'DATA_0811_1\\WW600WL100\\train',
#     result_num = 10,
#     roi_name = 'kidney', 
#     roi_name_chinese = '腎臟',
#     show_predicted_result = True,
#     show_segmentation_result = True, 
#     image_scale = 4)

## 隨機印出幾張CT影像的預測結果(測試集)

In [ ]:
from evaluate import *
show_result(
    target_dataset_base_dir = 'DATA_0811_1\\WW600WL100\\test',
    result_num = 10,
    roi_name = 'kidney', 
    roi_name_chinese = '腎臟',
    show_predicted_result = True,
    show_segmentation_result = True, 
    image_scale = 4)

## 第二次建模

資料集：DATA_0811_2
* **腫瘤視為正常腎臟**
* 去除case180、203，重新調整case160的影像大小
* 沒有做退縮
* WW = 600, WL = 100
* Base Model: Resnet-50
* Segmentation Model: U-Net

### 模型參數設定

In [9]:
dataset_dir = 'DATA_0811_2\\WW600WL100'
input_size = 512
epochs = 100
steps_per_epoch = 512

### 訓練及評估模型

目前用來評估模型表現的指標：
* average dice score per case
* average recall
* average precision
* global dice score

In [10]:
model_dir = os.path.join(dataset_dir, 'model_resnet50_unet')

if not os.path.exists(model_dir):
    os.makedirs(model_dir)
    print('-----建立新資料夾：' + model_dir + '-----') 

st = time.time()

target_dataset = dataset_dir
print(f'########## 即將用該資料集進行訓練：{target_dataset} ##########')

model_name = 'resnet50_unet_padding0'
checkpoints_path = os.path.join(model_dir, model_name)

keras.backend.clear_session() # 銷毀當前的TF，避免舊模型/網路層混亂，並減緩內存
model = resnet50_unet(n_classes = 2, input_height = input_size, input_width = input_size)

model.train(
    train_images = os.path.join(dataset_dir, 'train', 'images'),
    train_annotations = os.path.join(dataset_dir, 'train', 'annotations_tumor as kidney'),
    epochs = epochs, # default = 5
    steps_per_epoch = steps_per_epoch, # default = 512
    verify_dataset = False, # 當資料量很多時檢驗資料集會花很多時間
    checkpoints_path = checkpoints_path
)

ed = time.time()
spend_time = ed - st
print('花費時間(秒)：' + str(spend_time))
print()

-----建立新資料夾：DATA_0811_2\WW600WL100\model_resnet50_unet-----
########## 即將用該資料集進行訓練：DATA_0811_2\WW600WL100 ##########
Epoch 1/100
512/512 [==============================] - 186s 363ms/step - loss: 0.1013 - accuracy: 0.9723
saved  DATA_0811_2\WW600WL100\model_resnet50_unet\resnet50_unet_padding0.0
Epoch 2/100
512/512 [==============================] - 161s 315ms/step - loss: 0.0151 - accuracy: 0.9955
saved  DATA_0811_2\WW600WL100\model_resnet50_unet\resnet50_unet_padding0.1
Epoch 3/100
512/512 [==============================] - 161s 315ms/step - loss: 0.0094 - accuracy: 0.9971
saved  DATA_0811_2\WW600WL100\model_resnet50_unet\resnet50_unet_padding0.2
Epoch 4/100
512/512 [==============================] - 161s 315ms/step - loss: 0.0070 - accuracy: 0.9979
saved  DATA_0811_2\WW600WL100\model_resnet50_unet\resnet50_unet_padding0.3
Epoch 5/100
512/512 [==============================] - 161s 315ms/step - loss: 0.0055 - accuracy: 0.9982
saved  DATA_0811_2\WW600WL100\model_resnet50_unet\resnet50

512/512 [==============================] - 161s 315ms/step - loss: 0.0012 - accuracy: 0.9995
saved  DATA_0811_2\WW600WL100\model_resnet50_unet\resnet50_unet_padding0.44
Epoch 46/100
512/512 [==============================] - 161s 315ms/step - loss: 0.0013 - accuracy: 0.9995
saved  DATA_0811_2\WW600WL100\model_resnet50_unet\resnet50_unet_padding0.45
Epoch 47/100
512/512 [==============================] - 161s 315ms/step - loss: 0.0011 - accuracy: 0.9996
saved  DATA_0811_2\WW600WL100\model_resnet50_unet\resnet50_unet_padding0.46
Epoch 48/100
512/512 [==============================] - 161s 315ms/step - loss: 0.0010 - accuracy: 0.9996
saved  DATA_0811_2\WW600WL100\model_resnet50_unet\resnet50_unet_padding0.47
Epoch 49/100
512/512 [==============================] - 161s 315ms/step - loss: 0.0010 - accuracy: 0.9996
saved  DATA_0811_2\WW600WL100\model_resnet50_unet\resnet50_unet_padding0.48
Epoch 50/100
512/512 [==============================] - 161s 315ms/step - loss: 0.0012 - accuracy: 0.99

512/512 [==============================] - 161s 315ms/step - loss: 8.0838e-04 - accuracy: 0.9997
saved  DATA_0811_2\WW600WL100\model_resnet50_unet\resnet50_unet_padding0.88
Epoch 90/100
512/512 [==============================] - 161s 315ms/step - loss: 7.7408e-04 - accuracy: 0.9997
saved  DATA_0811_2\WW600WL100\model_resnet50_unet\resnet50_unet_padding0.89
Epoch 91/100
512/512 [==============================] - 161s 315ms/step - loss: 8.0468e-04 - accuracy: 0.9997
saved  DATA_0811_2\WW600WL100\model_resnet50_unet\resnet50_unet_padding0.90
Epoch 92/100
512/512 [==============================] - 161s 315ms/step - loss: 9.0545e-04 - accuracy: 0.9996
saved  DATA_0811_2\WW600WL100\model_resnet50_unet\resnet50_unet_padding0.91
Epoch 93/100
512/512 [==============================] - 161s 315ms/step - loss: 8.2474e-04 - accuracy: 0.9997
saved  DATA_0811_2\WW600WL100\model_resnet50_unet\resnet50_unet_padding0.92
Epoch 94/100
512/512 [==============================] - 161s 315ms/step - loss: 7.6

In [11]:
from evaluate import *

st = time.time()

result = evaluate_model(
    image_dir = os.path.join(dataset_dir, 'test', 'images'), 
    label_dir = os.path.join(dataset_dir, 'test', 'annotations_tumor as kidney'), 
    checkpoints_path = 'DATA_0811_2\\WW600WL100\\model_resnet50_unet\\resnet50_unet_padding0',
    calculate_predicting_indicators = True,
    output_predicted_result = False, 
    segment_out_predicted_region_from_original_images = False, 
    roi_name = 'kidney', 
    preds = None)

print('訓練集預測結果：')
print(f'average Dice score per case of kidney: {result[0]: .4f}')
print(f'average recall of kidney: {result[1]: .4f}')
print(f'average precision of kidney: {result[2]: .4f}')
print(f'global dice score of kidney: {result[3]: .4f}')
result_list.append(result)

ed = time.time()
spend_time = ed - st
print('花費時間(秒)：' + str(spend_time))
print()

----------生成模型預測結果----------
loaded weights  DATA_0811_2\WW600WL100\model_resnet50_unet\resnet50_unet_padding0.99


100%|████████████████████████████████████████████████████████████████████████████| 12656/12656 [13:35<00:00, 15.52it/s]


----------開始計算各項預測指標----------
目前進度：第500張照片
目前進度：第1000張照片
目前進度：第1500張照片
目前進度：第2000張照片
目前進度：第2500張照片
目前進度：第3000張照片
目前進度：第3500張照片
目前進度：第4000張照片
目前進度：第4500張照片
目前進度：第5000張照片
目前進度：第5500張照片
目前進度：第6000張照片
目前進度：第6500張照片
目前進度：第7000張照片
目前進度：第7500張照片
目前進度：第8000張照片
目前進度：第8500張照片
目前進度：第9000張照片
目前進度：第9500張照片
目前進度：第10000張照片
目前進度：第10500張照片
目前進度：第11000張照片
目前進度：第11500張照片
目前進度：第12000張照片
目前進度：第12500張照片
total case number: 12656
訓練集預測結果：
average Dice score per case of kidney:  0.9608
average recall of kidney:  0.9543
average precision of kidney:  0.9698
global dice score of kidney:  0.9542
花費時間(秒)：1515.652453660965



In [ ]:
# st = time.time()

# preds = result[4]

# _ = evaluate_model(
#     image_dir = os.path.join(dataset_dir, 'test', 'images'), 
#     label_dir = os.path.join(dataset_dir, 'test', 'annotations_tumor as kidney'), 
#     checkpoints_path = 'DATA_0811_2\\WW600WL100\\\model_resnet50_unet\\resnet50_unet_padding0',
#     calculate_predicting_indicators = False,
#     output_predicted_result = True, 
#     segment_out_predicted_region_from_original_images = True, 
#     roi_name = 'kidney', 
#     preds = preds
# )

# ed = time.time()
# spend_time = ed - st
# print('花費時間(秒)：' + str(spend_time))
# print()